In [105]:
import numpy as np
from scipy.spatial.transform import Rotation as R
import cv2

17. (5 баллов) Первая камера находится в начале координат, ее оптическая ось направлена вдоль оси `z`.     
Вторая камера сдвинута относительно первой на `1` в направлении оси `x`. Матрицы внутренних параметров обеих камер единичные.    
Сгенерировать случайный набор точек в трехмерном пространстве с `z>0`, спроектировать их на обе камеры(`cv2.projectPoints`), по проекциям найти необходимую матрицу(`cv2.findEssentialMat`).

In [106]:
k = np.eye(3)

rt2 = np.c_[R.from_rotvec([0,0,0]).as_matrix(), np.array([1, 0, 0])]

def gene_points():
    points = []
    for i in range(100):
        point = [np.random.choice(np.arange(100), 3),
                 np.random.choice(np.arange(100), 3),
                 np.random.choice(np.arange(start=1, stop=100), 3)
            	]
        points.append(point)
    return np.array(points, dtype=np.float64)

In [107]:
points = gene_points().T.reshape(-1,1,3)

In [108]:
image_points1, jac1 = cv2.projectPoints(points, np.zeros(3), np.zeros(3), np.eye(3), None)

In [109]:
image_points2, jac2 = cv2.projectPoints(points, np.zeros(3), np.array([1, 0, 0], dtype=np.float64), np.eye(3), None)

In [110]:
def fixing(array):
    array_temp = []
    for i in range(len(array)):
        array_temp.append(array[i][0])
    return np.array(array_temp, dtype=np.float64)

image_points1 = fixing(image_points1)
image_points2 = fixing(image_points2)

In [111]:

pts_1_norm = cv2.undistortPoints(np.expand_dims(image_points1, axis=1), cameraMatrix=np.eye(3), distCoeffs=None)
pts_2_norm = cv2.undistortPoints(np.expand_dims(image_points2, axis=1), cameraMatrix=np.eye(3), distCoeffs=None)

E, mask = cv2.findEssentialMat(pts_1_norm, pts_2_norm, focal=1.0, pp=(0., 0.), method=cv2.RANSAC, prob=0.999, threshold=3.0)
E

array([[-1.38304023e-15,  1.21917520e-13, -6.27710474e-13],
       [-1.22175957e-13, -5.32016205e-16, -7.07106781e-01],
       [ 6.36594236e-13,  7.07106781e-01,  1.16094618e-15]])

18. По необходимой матрице, полученной в задаче `18`, получить позу первой камеры относительно второй.    
Для этого использовать `cv2.decomposeEssentialMat`, и из получившихся `4` решений выбрать одно, используя условие, что трехмерные точки должны быть перед камерами.

In [112]:
R1, R2, t = cv2.decomposeEssentialMat(E)

In [113]:
t = fixing(t)

In [116]:
D = np.c_[R1, t]
D

array([[ 1.00000000e+00, -1.78799687e-12, -3.45200305e-13,
         1.00000000e+00],
       [-1.78799687e-12, -1.00000000e+00, -3.88578058e-16,
        -8.87716666e-13],
       [-3.45200305e-13,  4.44089210e-16, -1.00000000e+00,
        -1.72417410e-13]])

19. (5 баллов) Найти матрицу внутренних параметров камеры изображения    
 https://drive.google.com/file/d/1mC0PI9k4q_wJt9iAn6uosEVSJb9PcZIk/view?usp=sharing     
 предполагая, что оптическая ось проходит строго через центр изображения, а фокусное расстояние в эквиваленте `35мм` равно `0.64` дюйма.     
 Почему результат отличается от параметров, полученных методом калибрации камеры?    
 https://drive.google.com/file/d/1A4H84PLy7971Xd1ErS1bRRupWk9_TCYI/view?usp=sharing